## 地点推荐系统

## 读取数据

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
datasrc = './data/mappedpoiinfo.txt'
dataset = pd.read_table(datasrc, header = None, delim_whitespace = True, names = ['ID', 'latitude', 'longtitude', 'city', 'subclass1', 'subclass2'])

In [3]:
dataset.head()

,ID,latitude,longtitude,city,subclass1,subclass2
0,107780,22.297428,114.172607,overseas,公共机构/办公/住宅,公寓/小区/里弄
1,70990,31.136839,121.422639,shanghai,NaN,NaN
2,132379,31.232181,121.397632,shanghai,NaN,NaN
3,38132,34.266345,117.187751,xuzhou,餐饮,咖啡馆
4,104522,27.700000,85.333333,overseas,NaN,NaN
